In [0]:
import os
import json

In [8]:
if (not os.path.exists('yelp_academic_dataset_business.json')) or (not os.path.exists('yelp_academic_dataset_review.json')):
    kaggle_path = os.path.expanduser('~/.kaggle')
    kaggle_json_path = os.path.join(kaggle_path, 'kaggle.json')
    if not os.path.exists(kaggle_json_path):
        from getpass import getpass
        kaggle_json = getpass('Insert kaggle.json:')
        os.makedirs(kaggle_path, exist_ok=True)
        with open(kaggle_json_path, 'w') as f:
            f.write(kaggle_json)
        os.chmod(kaggle_json_path, 0o600)
    !kaggle datasets download yelp-dataset/yelp-dataset
    !unzip yelp-dataset.zip yelp_academic_dataset_business.json yelp_academic_dataset_review.json
    !rm yelp-dataset.zip

Insert kaggle.json:··········
100% 4.48G/4.48G [01:27<00:00, 70.0MB/s]
100% 4.48G/4.48G [01:27<00:00, 55.0MB/s]
Archive:  yelp-dataset.zip
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_review.json  


In [16]:
pizza_businesses = []
with open('yelp_academic_dataset_business.json', 'r') as f:
    for line in f:
        line_json = json.loads(line)
        if 'Pizza' in (line_json.get('categories') or ''):
            pizza_businesses.append(line_json)
for i in pizza_businesses:
  print(i)


{'business_id': 'ZkzutF0P_u0C0yTulwaHkA', 'name': 'Lelulos Pizzeria', 'address': '311 Unity Center Rd', 'city': 'Plum', 'state': 'PA', 'postal_code': '15239', 'latitude': 40.4899958, 'longitude': -79.7792879, 'stars': 4.0, 'review_count': 31, 'is_open': 1, 'attributes': {'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'True', 'OutdoorSeating': 'False', 'HasTV': 'True', 'WiFi': "'no'", 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'RestaurantsGoodForGroups': 'True', 'RestaurantsTakeOut': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsReservations': 'True', 'BikeParking': 'True', 'RestaurantsDelivery': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", 'NoiseLevel': "'quiet'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

